<a href="https://colab.research.google.com/github/denis911/image-classification-transfer-learning-pytorch-keras-comparison/blob/main/HW8_hair_type_classification_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download the dataset

In [1]:
!nvidia-smi # nvidia system information for T4 Tesla GPU

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
# We'll build a model for classifying various hair types. For this, we will use the Hair Type dataset that was obtained from Kaggle and slightly rebuilt.

# RUN ONCE:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip;

--2025-12-01 20:40:11--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-01T21%3A27%3A41Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-01T20%3A27%3A23Z&ske=2025-12-01T21%3A27%3A41Z&sks=b&skv=2018-11-09&sig=SxqcQTPcTu74bB0rCkfD5qRH1qEgGyClHR2%2FlsYDPr4%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDYyMzQxMSwibmJmIjoxNzY0NjIxNjExLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5i

In [3]:
# RUN ONCE
# unzip dataset into Colab home folder and check:
!unzip data.zip > /dev/null
# > /dev/null to prohibit long output, then check files manually